In [1]:
import numpy as np
from util import load_dataset

In [2]:
train, labels = load_dataset('./data/train.csv')

In [3]:
import tensorflow as tf

In [4]:
from keras.applications.resnet50 import preprocess_input

train = preprocess_input(train)

Using TensorFlow backend.


In [5]:
images = np.zeros((train.shape[0], train.shape[1], train.shape[2], 3))
images.shape

(42000, 28, 28, 3)

In [6]:
images[:,:,:,1] = train

In [7]:
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers import Dense, Lambda, Input
from keras.models import Model
from keras.backend import tf as ktf

In [13]:
mobilenet = MobileNet(weights = 'imagenet')
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [14]:
for _ in range(5):
    mobilenet.layers.pop()
mobilenet.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [15]:
# for layer in resnet.layers:
#     layer.trainable = False
inp = Input(shape=(28,28,3,))
resize_layer = Lambda(lambda image: ktf.image.resize_images(image, (244, 244)))(inp)
mobilenet = mobilenet(resize_layer)
out = Dense(10, activation='softmax')(mobilenet)
model = Model(inp, out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 244, 244, 3)       0         
_________________________________________________________________
mobilenet_1.00_224 (Model)   (None, 1000)              3228864   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 3,238,874
Trainable params: 3,216,986
Non-trainable params: 21,888
_________________________________________________________________


In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [17]:
batch_size=16

In [19]:
model.fit(images, labels, validation_split=0.1, epochs=3, batch_size=batch_size)

Train on 37800 samples, validate on 4200 samples
Epoch 1/3
37800/37800 [==============================] - 521s 14ms/step - loss: 0.4735 - acc: 0.7753 - val_loss: 0.9252 - val_acc: 0.6626
Epoch 2/3
37800/37800 [==============================] - 519s 14ms/step - loss: 0.3935 - acc: 0.8016 - val_loss: 0.3681 - val_acc: 0.7990
Epoch 3/3
37800/37800 [==============================] - 519s 14ms/step - loss: 0.3117 - acc: 0.8600 - val_loss: 1.1869 - val_acc: 0.6162
